## Test PyTorch Workflow

This notebook is supposed to be used to test the PyTorch workflow before training we test a proper model.

In [1]:
import time
import datetime
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

np.random.seed(0)
program_start = time.time()
program_start_date = datetime.datetime.now()

In [2]:
def simulate_data(n_samples, n_features):
    """Simulate linear model with 3 relevant features."""
    weights = np.array([-5, 10, 4])

    x = np.random.uniform(size=(n_samples, n_features))
    error = np.random.normal(size=(n_samples,))

    y = x[:, :3] @ weights + error

    return x, y

In [3]:
# data = simulate_data(n_samples=10_000, n_features=100_000)

data = simulate_data(n_samples=10_000, n_features=500)

x_train, x_test, y_train, y_test = train_test_split(
    *data, test_size=0.25, random_state=0
)

### PyTorch Network

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from tqdm import tqdm

In [5]:
class NeuralNetwork(nn.Module):
    """Neural network class.

    Is called with input dimension (input_dim) and a list of integers,
    where each entry corresponds to the number of nodes in the respective
    layer.

    """

    def __init__(self, input_dim, hidden_dim):
        super(NeuralNetwork, self).__init__()

        self.input_dim = input_dim
        self.hidden_dim = hidden_dim

        current_dim = input_dim
        self.layers = nn.ModuleList()
        for hdim in hidden_dim:
            self.layers.append(nn.Linear(current_dim, hdim))
            current_dim = hdim
        self.layers.append(nn.Linear(current_dim, 1))

    def forward(self, x):
        for layer in self.layers[:-1]:
            x = F.relu(layer(x))
        out = self.layers[-1](x)
        return out

## Training

In [6]:
x_train = torch.tensor(x_train, dtype=torch.float)
x_test = torch.tensor(x_test, dtype=torch.float)
y_train = torch.tensor(y_train.reshape(-1, 1), dtype=torch.float)

In [7]:
loss_function = nn.MSELoss()

In [8]:
input_dim = x_test.shape[1]

# hidden_dim = 25 * [input_dim]
hidden_dim = [3]  # results in 1 hidden layer with 3 nodes

model = NeuralNetwork(input_dim, hidden_dim)

In [9]:
optimizer = optim.Adam(model.parameters(), lr=0.005)

In [10]:
n_epochs = 10_000

for i in tqdm(range(n_epochs)):
    # initialize the gradient of model parameters
    optimizer.zero_grad()

    # calculate the loss
    y_val = model(x_train)
    loss = loss_function(y_val, y_train)

    # backpropagation
    loss.backward()

    # update parameters
    optimizer.step()

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:57<00:00, 175.28it/s]


## Validation

In [11]:
y_pred = model(x_test).cpu().detach().numpy()

mean_squared_error(y_pred, y_test)  # should be around 1 because of the error variance

1.0972190175643948

## Runtime

In [12]:
program_end = time.time()

f"Program runtime: {int(program_end - program_start)} seconds"

'Program runtime: 57 seconds'